# Playing with Spark and Taxi Data

## Configure our Spark Session

In [2]:
// Import GeoSpark Stuff
import org.datasyslab.geosparksql.utils.GeoSparkSQLRegistrator
import org.datasyslab.geosparksql.utils.Adapter
import org.datasyslab.geosparksql.UDF.UdfRegistrator
import org.datasyslab.geosparksql.UDT.UdtRegistrator
import org.datasyslab.geospark.serde.GeoSparkKryoRegistrator
import org.apache.spark.sql.geosparksql.strategy.join.JoinQueryDetector

// Import SparkStuff
import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.sql.functions._

In [3]:
var spark = SparkSession
  .builder
  .appName("NYCTaxis")
  .config("spark.serializer", classOf[KryoSerializer].getName)
  .config("spark.kryo.registrator", classOf[GeoSparkKryoRegistrator].getName)
  .enableHiveSupport
  .getOrCreate()
GeoSparkSQLRegistrator.registerAll(spark)

spark = org.apache.spark.sql.SparkSession@1dfb062f


org.apache.spark.sql.SparkSession@1dfb062f

## Get Taxi Data 

The taxi data is a part of Amazon's open datasets so we can just use their bucket location. For now I will only inspect 2017 datasets.

In [5]:
val tripData = spark
    .read
    .option("header", "true")
    .csv("s3://nyc-tlc/trip data/yellow_tripdata_2017-*.csv")

tripData = [VendorID: string, tpep_pickup_datetime: string ... 15 more fields]


[VendorID: string, tpep_pickup_datetime: string ... 15 more fields]

In [6]:
%%dataframe
tripData

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2017-01-09 11:13:28,2017-01-09 11:25:45,1,3.30,1,N,263,161,1,12.5,0,0.5,2,0,0.3,15.3
1,2017-01-09 11:32:27,2017-01-09 11:36:01,1,.90,1,N,186,234,1,5,0,0.5,1.45,0,0.3,7.25
1,2017-01-09 11:38:20,2017-01-09 11:42:05,1,1.10,1,N,164,161,1,5.5,0,0.5,1,0,0.3,7.3
1,2017-01-09 11:52:13,2017-01-09 11:57:36,1,1.10,1,N,236,75,1,6,0,0.5,1.7,0,0.3,8.5
2,2017-01-01 00:00:00,2017-01-01 00:00:00,1,.02,2,N,249,234,2,52,0,0.5,0,0,0.3,52.8
1,2017-01-01 00:00:02,2017-01-01 00:03:50,1,.50,1,N,48,48,2,4,0.5,0.5,0,0,0.3,5.3
2,2017-01-01 00:00:02,2017-01-01 00:39:22,4,7.75,1,N,186,36,1,22,0.5,0.5,4.66,0,0.3,27.96
1,2017-01-01 00:00:03,2017-01-01 00:06:58,1,.80,1,N,162,161,1,6,0.5,0.5,1.45,0,0.3,8.75
1,2017-01-01 00:00:05,2017-01-01 00:08:33,2,.90,1,N,48,50,1,7,0.5,0.5,0,0,0.3,8.3
2,2017-01-01 00:00:05,2017-01-01 00:05:04,5,1.76,1,N,140,74,2,7,0.5,0.5,0,0,0.3,8.3


In [7]:
val tripMonths = tripData
    .select($"tpep_pickup_datetime")
    .withColumn("tripMonth", month($"tpep_pickup_datetime"))

tripMonths = [tpep_pickup_datetime: string, tripMonth: int]


[tpep_pickup_datetime: string, tripMonth: int]

I've always been curious as which months are the busiest for taxis and ubers, and now I can an idea from the taxi data.

In [8]:
%%dataframe --limit 12
tripMonths.groupBy($"tripMonth").count.sort($"count".desc)

tripMonth,count
3,10294633
5,10102128
4,10046197
10,9768740
1,9710231
6,9656993
12,9508086
11,9284778
2,9168831
9,8945574


OK, now let's get location data to find the busiest zone. I download the GeoJSON file from [here](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc).

In [9]:
val jsonFile = spark
    .read
    .option("multiline",true)
    .json("s3://jamie-curtis/datasets/NYC Taxi Zones.geojson")

jsonFile = [features: array<struct<geometry:struct<coordinates:array<array<array<array<double>>>>,type:string>,properties:struct<borough:string,location_id:string,objectid:string,shape_area:string,shape_leng:string,zone:string>,type:string>>, type: string]


[features: array<struct<geometry:struct<coordinates:array<array<array<array<double>>>>,type:string>,properties:struct<borough:string,location_id:string,objectid:string,shape_area:string,shape_leng:string,zone:string>,type:string>>, type: string]

In [10]:
jsonFile.printSchema

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- geometry: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- properties: struct (nullable = true)
 |    |    |    |-- borough: string (nullable = true)
 |    |    |    |-- location_id: string (nullable = true)
 |    |    |    |-- objectid: string (nullable = true)
 |    |    |    |-- shape_area: string (nullable = true)
 |    |    |    |-- shape_leng: string (nullable = true)
 |    |    |    |-- zone: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [11]:
val locationDS = jsonFile
    .select(explode($"features").as("f"))
    .select($"f.geometry.type"
           ,$"f.properties.*"
           ,callUDF("ST_GeomFromGeoJson", to_json($"f.geometry")).as("geom_spark"))

locationDS = [type: string, borough: string ... 6 more fields]


[type: string, borough: string ... 6 more fields]

In [13]:
%%dataframe --limit 1
locationDS

type borough location_id objectid shape_area shape_leng zone geom_spark MultiPolygon EWR 1 1 0.0007823067885 0.116357453189 Newark Airport POLYGON ((-74.18445299999996 40.694995999999904, -74.18448899999999 40.69509499999987, -74.18449799999996 40.69518499999987, -74.18438099999997 40.69587799999989, -74.18428199999994 40.6962109999999, -74.18402099999997 40.697074999999884, -74.18391299999996 40.69750699999986, -74.18375099999997 40.69779499999988, -74.18363399999998 40.6983259999999, -74.18356199999994 40.698451999999875, -74.18354399999998 40.69855999999988, -74.18350799999996 40.69870399999992, -74.18327399999998 40.70008999999988, -74.18315699999994 40.701214999999884, -74.18316599999997 40.702384999999886, -74.18313899999998 40.7026279999999, -74.18309399999998 40.7028529999999, -74.18299499999995 40.70315899999985, -74.18284199999994 40.70346499999989, -74.18264399999998 40.70373499999988, -74.18242799999996 40.70395099999992, -74.18220299999996 40.704139999999896, -74.18203199999994 40.70425699999987, -74.18180699999994 40.7043919999999, -74.18157299999996 40.70449999999988, -74.18132099999997 40.70460799999991, -74.18080799999996 40.7047879999999, -74.179467 40.70534599999992, -74.17887299999995 40.70554399999987, -74.17831499999994 40.70572399999987, -74.17776599999996 40.70589499999988, -74.17709099999996 40.706092999999896, -74.17699199999998 40.70613799999988, -74.17689299999995 40.70619199999988, -74.17664999999994 40.70641699999988, -74.17642499999994 40.706695999999916, -74.17628999999994 40.70689399999988, -74.17608299999995 40.70710999999989, -74.17599299999995 40.70719099999991, -74.17589399999997 40.707262999999905, -74.17565999999994 40.70737999999988, -74.17538099999996 40.707469999999915, -74.17515599999996 40.707514999999894, -74.17475999999994 40.707595999999924, -74.17417499999993 40.70766799999991, -74.17388699999998 40.70773099999992, -74.17347299999994 40.707748999999865, -74.17275299999994 40.707802999999906, -74.17188899999996 40.707910999999854, -74.17163699999998 40.70795599999986, -74.17133999999999 40.707964999999895, -74.17120499999999 40.70795599999986, -74.16994499999998 40.707973999999886, -74.16888299999994 40.7079379999999, -74.16681299999993 40.70785699999989, -74.16442799999999 40.70779399999987, -74.16401399999995 40.70777599999992, -74.16233999999997 40.707721999999876, -74.16081899999995 40.70764099999991, -74.16057599999993 40.70760499999988, -74.16033299999998 40.70756899999987, -74.160063 40.7074879999999, -74.15938799999998 40.707262999999905, -74.15904599999999 40.707145999999916, -74.15891999999997 40.70710999999989, -74.15827199999995 40.70687599999993, -74.15459099999998 40.705651999999894, -74.15409599999998 40.70544499999989, -74.15401499999997 40.70538199999988, -74.15387999999996 40.705327999999895, -74.15376299999997 40.705408999999875, -74.15323199999995 40.70524699999987, -74.15317799999997 40.70531899999989, -74.15306999999996 40.7052829999999, -74.15359199999995 40.70437399999987, -74.15386199999995 40.7038429999999, -74.15513999999996 40.70155699999987, -74.15544599999998 40.70108899999988, -74.15575199999995 40.7006659999999, -74.15600399999994 40.70026099999991, -74.15635499999996 40.69975699999986, -74.15745299999998 40.69809199999988, -74.15754299999998 40.6979389999999, -74.15758799999998 40.69781299999988, -74.15762399999994 40.69767799999991, -74.15829899999994 40.696705999999885, -74.15951399999994 40.69488799999988, -74.15958599999993 40.69476199999984, -74.16014399999995 40.69410499999988, -74.16057599999993 40.693222999999875, -74.16262799999998 40.69028899999989, -74.16279899999995 40.69002799999989, -74.16290699999996 40.68987499999987, -74.16292499999997 40.689874999999866, -74.16295199999996 40.689874999999866, -74.16306899999995 40.68989299999988, -74.16309599999994 40.689928999999886, -74.16322199999996 40.68998299999989, -74.16331199999996 40.68999199999993, -74.16341099999994 40.69000099999988, -74.16352799999999 40.69000999999986, -74.163806999

In [14]:
val tripLocations = tripData
    .join(locationDS.select($"location_id", $"zone".as("pick_up_zone"))
         , $"PULocationID" === $"location_id"
         , "inner").drop($"location_id")
    .join(locationDS.select($"location_id", $"zone".as("drop_off_zone"))
         , $"DOLocationID" === $"location_id"
         , "inner").drop($"location_id")

tripLocations = [VendorID: string, tpep_pickup_datetime: string ... 17 more fields]


[VendorID: string, tpep_pickup_datetime: string ... 17 more fields]

OK, let's see the 10 most popular pick up locations

In [15]:
%%dataframe
tripLocations.groupBy("pick_up_zone").count.sort($"count".desc)

pick_up_zone,count
Upper East Side South,4437101
Midtown Center,4264964
Upper East Side North,4019431
Penn Station/Madison Sq West,3962805
Midtown East,3856212
Times Sq/Theatre District,3830469
Union Sq,3696889
Murray Hill,3625208
Clinton East,3555159
East Village,3322177


In [16]:
%%dataframe
tripLocations.groupBy("drop_off_zone").count.sort($"count".desc)

drop_off_zone,count
Midtown Center,4187572
Upper East Side North,4142175
Upper East Side South,3899475
Murray Hill,3610257
Times Sq/Theatre District,3512881
Midtown East,3442198
Union Sq,3164130
Penn Station/Madison Sq West,3082515
Clinton East,3067502
Lincoln Square East,2923849


OK, what was the average tip in 2017.

In [17]:
tripData.select(mean($"tip_amount").as("avg_tip_amount")).show

+-----------------+
|   avg_tip_amount|
+-----------------+
|1.837307255792978|
+-----------------+



In [18]:
tripData.groupBy($"payment_type").agg(mean($"tip_amount").as("avg_tip_amount")).sort($"avg_tip_amount".desc).show

+------------+--------------------+
|payment_type|      avg_tip_amount|
+------------+--------------------+
|           1|   2.735025633448391|
|           2|2.809800210321084...|
|           5|                 0.0|
|           3|-0.00148715945382395|
|           4|-0.00442512393655...|
+------------+--------------------+



How about if we filter out people who don't tip?

In [19]:
tripData.where($"tip_amount" > 0).select(mean($"tip_amount").as("avg_tip_amount")).show

+------------------+
|    avg_tip_amount|
+------------------+
|2.9334217641378317|
+------------------+



OK, what percentage of rides actually result in a tip?

In [20]:
val nRides = tripData.count
tripData
  .withColumn("tiped", when($"tip_amount" <= 0, "No Tip").otherwise("Tip"))
  .groupBy($"tiped")
  .count
  .withColumn("freq", $"count"/nRides)
  .show

+------+--------+-------------------+
| tiped|   count|               freq|
+------+--------+-------------------+
|   Tip|70466546| 0.6208677253965602|
|No Tip|43030328|0.37913227460343973|
+------+--------+-------------------+



nRides = 113496874


113496874

In [21]:
spark.sql("show databases").show

+------------+
|databaseName|
+------------+
|     default|
|   num_facts|
+------------+

